In [1]:
import sys
import os
import pandas as pd
import numpy as np
from razdel import sentenize

from tqdm._tqdm_notebook import tqdm_notebook, tqdm

Данные: решила оставить то, что насобирала и наразмечала. Три класса: unrelated, journalism и clickbait. Попробовала ROUGE, остальные метрики неоч для первой итерации, они вообще из других тематик (CIDEr - компютервижн, расстояние Левенштейна - ЗАЧЕМ это же редакторское расстояние, оно не отражает семантику текста, имхо, BLEU сравнивает человеческий перевод с машинным, тоже мимо). <br/>
Модель: будем фигачить лстм по образу и подобию того, что накатали португальцы

In [2]:
import ipywidgets as widgets
from ipywidgets import interact

In [3]:
from text_preprocessing import Preprocessor
from plot_tools import plot_cm

In [4]:
with open("../data/external/stopwords.txt", mode="r") as f:
    stopwords = [s.replace("\n", "") for s in f.readlines()]
    
stopwords.remove("не") # несильно влияет на качество, но для отдельных примеров мб решающим
stopwords.remove("нет")

p = Preprocessor(stopwords=stopwords)

In [5]:
data = pd.read_json("../data/interim/training-gbc-normalized-02-04.json", orient='records', lines=True)

In [6]:
@interact
def show_by_source(source=data.source.unique()):
    df=data
    print("Total number of posts: {}".format(df.shape[0]))
    print("Number of posts from source {0}: {1}".format(source, df[df["source"]==source].shape[0]))
    return df[df["source"]==source].head(10)

interactive(children=(Dropdown(description='source', options=('interfax.ru', 'dni.ru', 'utro.ru', 'artificial'…

In [7]:
twenty_texts = data.text.tolist()[:20]
twenty_titles = data.title.tolist()[:20]

In [8]:
get_lead = lambda body : " ".join([substring.text for substring in list(sentenize(body))[:2]]) 

In [9]:
tqdm_notebook.pandas(desc="Extracting lead")

data["lead_sentence"] = data["text"].progress_apply(get_lead)

In [10]:
tqdm_notebook.pandas(desc="Normalizing lead")

data["lead_norm"] = data["lead_sentence"].progress_apply(lambda x : p.beautify(x, normalize_text=True))

In [11]:
from rouge import Rouge
rouge = Rouge()

In [12]:
rouge.get_scores(data.title_norm.values[0], data.lead_norm.values[0])

[{'rouge-1': {'f': 0.31249999736328127, 'p': 1.0, 'r': 0.18518518518518517},
  'rouge-2': {'f': 0.2580645138813736, 'p': 1.0, 'r': 0.14814814814814814},
  'rouge-l': {'f': 0.1903271405493217, 'p': 1.0, 'r': 0.18518518518518517}}]

In [13]:
rouge.get_scores(p.beautify("В Сочи пройдет зимняя Олимпиада", normalize_text=True),
                 p.beautify("Спортивные соревнования предложили вновь провести на российском курорте \
                 В 2026 г. на Красной Поляне могут пройти олимпийские соревнования по бобслею,\
                 скелетону и санному спорту.", normalize_text=True)) 

[{'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}]

In [14]:
p.beautify("Спортивные соревнования предложили вновь провести на российском курорте \
                 В 2026 г. на Красной Поляне могут пройти олимпийские соревнования по бобслею,\
                 скелетону и санному спорту.", normalize_text=True)

'спортивный соревнование предлагать вновь проводить российский курорт красный поляна мочь проходить олимпийский соревнование бобслей скелетон санный спорт'

In [15]:
rouge.get_scores(p.beautify("Суд в США оставил обвиняемую в шпионаже россиянку под стражей из-за «риска побега» \
                 Россиянку Марию Бутину, задержанную в Вашингтоне по подозрению в шпионаже, оставили \
                 под стражей до начала судебного разбирательства по ее делу, однако адвокат Нил Дрисколл не сомневается, \
                 что ему удастся доказать невиновность своей подзащитной, и в итоге девушку освободят.", normalize_text=True),
                 p.beautify("Адвокат уверен, что он докажет невиновность Бутиной, и ее освободят", normalize_text=True))

[{'rouge-1': {'f': 0.22222221944444445,
   'p': 0.13333333333333333,
   'r': 0.6666666666666666},
  'rouge-2': {'f': 0.05263157666204997, 'p': 0.030303030303030304, 'r': 0.2},
  'rouge-l': {'f': 0.13756613756592023,
   'p': 0.13333333333333333,
   'r': 0.6666666666666666}}]

In [16]:
bait_head = "Звезду первого советского фильма ужасов убило проклятье"
bait_body = "В студенческие годы она напророчила Анатолию Ромашину трех жен, \
            столько же детей (двух дочерей и сына) и смерть в 83 года. \
            Сбылось все, кроме последнего. Актер погиб не старым – в 69 лет. \
            А виною всему, по мнению поклонников, – его роковые роли."

rouge.get_scores(p.beautify(bait_body, normalize_text=True),
                 p.beautify(bait_head, normalize_text=True)) 

[{'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}]

In [17]:
def get_rouge_scores(headlines, articles):
    rouge_vect = []
    fails = 0
    for h, a in tqdm(zip(headlines, articles)):
        rouge_values = []
        try:
            scores = rouge.get_scores(h, a)
            rouge_values += [scores[0]['rouge-1']['f']]
            rouge_values += [scores[0]['rouge-2']['f']]
            rouge_values += [scores[0]['rouge-l']['f']]
        except:
            rouge_values = [0,0,0]
            fails += 1
        rouge_vect += [rouge_values]
    print("Number of fails: {}".format(fails))
    return rouge_vect

In [18]:
data["head_lead_rouge"]= get_rouge_scores(data.title_norm.tolist(), data.lead_norm.tolist())

12575it [00:02, 4363.50it/s]

Number of fails: 0


In [20]:
data["head_body_rouge"]= get_rouge_scores(data.title_norm.tolist(), data.text_norm.tolist())

12575it [00:11, 1066.92it/s]

Number of fails: 0


In [21]:
data.head()

,id,label,source,text,text_norm,title,title_norm,url,lead_sentence,lead_norm,head_lead_rouge,head_body_rouge
0,124272138,Journalism,interfax.ru,Россия поставила Анголе четыре истреьителя Су-...,россия поставлять ангола четыре истреьитель ко...,Россия поставила Анголе четыре истребителя Су-...,россия поставлять ангола четыре истребитель,http://www.interfax.ru/world/638657,Россия поставила Анголе четыре истреьителя Су-...,россия поставлять ангола четыре истреьитель ко...,"[0.31249999736328127, 0.2580645138813736, 0.19...","[0.09708737771703273, 0.05925925868422497, 0.0..."
1,124272120,Journalism,interfax.ru,Нерезиденты и дочерние иностранные банки в окт...,нерезидент дочерний иностранный банк октябрь с...,Нерезиденты в октябре купили ОФЗ на аукционах ...,нерезидент октябрь купить офз аукцион минфин м...,http://www.interfax.ru/business/638658,Нерезиденты и дочерние иностранные банки в окт...,нерезидент дочерний иностранный банк октябрь с...,"[0.3030302993572085, 0.06249999658203144, 0.21...","[0.12727272592396696, 0.036809815128909656, 0...."
2,111233217,Journalism,interfax.ru,Северо-Кавказский окружной военный суд вынес о...,окружной военный суд вынести обвинительный при...,Житель Ростовской области осужден на 11 лет за...,житель ростовский область осуждать год подгото...,http://www.interfax.ru/russia/627809,Северо-Кавказский окружной военный суд вынес о...,окружной военный суд вынести обвинительный при...,"[0.19999999711250002, 0.05263157628808878, 0.0...","[0.102040815, 0.01904761797006809, 0.055245027..."
3,110816371,Journalism,interfax.ru,"Футболисты московского ""Динамо"" нанесли пораже...",футболист московский динамо наносить поражение...,"Динамо"" обыграло ""Оренбург"" в матче чемпионата...",динамо обыгрывать оренбург матч чемпионат росс...,http://www.interfax.ru/sport/627601,"Футболисты московского ""Динамо"" нанесли пораже...",футболист московский динамо наносить поражение...,"[0.2499999958680556, 0.0, 0.1929223744293267]","[0.13953488099513256, 0.0, 0.08585289048708412]"
4,121896641,Journalism,interfax.ru,Движение FEMEN взяло на себя ответственность з...,движение femen взять ответственность акция про...,Активистки FEMEN провели в Париже акцию против...,активистка femen проводить париж акция против ...,http://www.interfax.ru/world/637359,Движение FEMEN взяло на себя ответственность з...,движение femen взять ответственность акция про...,"[0.3749999965820313, 0.05882352650519045, 0.16...","[0.2105263136349646, 0.029850744638004103, 0.0..."


In [22]:
columns = ["id", "label", "source", "text", "title", "url"]
data[columns].to_json("../data/raw/final-raw-for-senior-thesis.json", force_ascii=False, orient='records', lines=True)

## Модель

In [24]:
from keras.layers import Bidirectional, Embedding, LSTM, Masking, Dense, Dropout
from keras.layers import GlobalMaxPooling1D, Input, TimeDistributed
from keras.layers import concatenate, multiply, subtract

from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.models import Model, load_model

max_sents = average number of sentences in article + 2\*sigma <br/>
max_seq_len = average length of sample concat(headlines, leads) + 2\*sigma

In [43]:
data["lead_len"] = data.lead_norm.apply(lambda x : len(x.split()))
data["text_sent_number"] = data.text.apply(lambda body : len([substring.text for substring in list(sentenize(body))]))

In [44]:
# get maximum sequence length
avg_lead = data.lead_len.mean()
std_lead = data.lead_len.std()
max_seq_len = int(round(avg_lead + 2*std_lead))

# get maximum number of sentences in text
avg_sents = data.text_sent_number.mean()
std_sents = data.text_sent_number.std()
max_sents = int(round(avg_sents + 2*std_sents))

In [42]:
max_seq_len

340

In [45]:
max_sents

29

In [46]:
input_headline = Input(shape=(max_seq_len,))
input_body = Input(shape=(max_sents, max_seq_len))
input_lead = Input(shape=(max_seq_len,))
input_overlap_body = Input(shape=(1,))
input_overlap_lead = Input(shape=(1,))
input_polarity_body = Input(shape=(2,))
input_polarity_lead = Input(shape=(2,))
input_hand_body = Input(shape=(26,))
input_hand_lead = Input(shape=(26,))
input_cos_body = Input(shape=(1,))
input_cos_lead = Input(shape=(1,))
input_rouge_body = Input(shape=(3,))
input_rouge_lead = Input(shape=(3,))

In [47]:
data.to_json("../data/interim/training-lstm-w-lengths-11-04.json", force_ascii=False, lines=True, orient='records')